<a href="https://colab.research.google.com/github/ahmadsaad007/Deep_Learning_Projects/blob/master/Image_Classification_Using_TPU%20Error%20File!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install --upgrade tensorflow==2.3
#!pip install --upgrade pip

     |████████████████████████████████| 320.4 MB 39 kB/s 
     |████████████████████████████████| 6.8 MB 59.7 MB/s 
     |████████████████████████████████| 459 kB 45.1 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [1]:
import numpy as np
from keras.models import Sequential
import os
from keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import *
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import Input

In [2]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

Found TPU at: grpc://10.57.168.34:8470


Below code does not work


```
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
```

Mount the Google Drive to obtain dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_path = "/content/drive/My Drive/Eye Disease Research/train"
test_path = "/content/drive/My Drive/Eye Disease Research/test"

Read the image files and split them into train, test and validation

In [5]:
imgdata = ImageDataGenerator( rescale=1.0/255.0, validation_split=0.20)
train_batches = imgdata.flow_from_directory(train_path,class_mode='categorical', subset='training', target_size=(768, 496) ,
                                            classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 32)
test_batches = imgdata.flow_from_directory(test_path, target_size=(768, 496) ,class_mode='categorical',
                                           classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 32)
valid_batches = imgdata.flow_from_directory(train_path, subset='validation', target_size=(768, 496) ,class_mode='categorical',
                                            classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 32)


Found 66824 images belonging to 4 classes.
Found 1001 images belonging to 4 classes.
Found 16705 images belonging to 4 classes.


In [6]:

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)


Running on TPU  ['10.57.168.34:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.57.168.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.57.168.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [8]:
def model_create():
  vgg16_model = VGG16(input_tensor=Input(shape=(768, 496, 3)),input_shape=(768,496,3), include_top = False)
  model= Sequential()
  model.add(vgg16_model)
 
  #Make the vgg16's layers non-trainable
  for layer in model.layers:
    layer.trainable = False
  model.add(Flatten())
  model.add(Dropout(0.2))
  model.add(Dense(512, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  return model

In [9]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(train_batches.classes), 
                                                  train_batches.classes)
class_weights

array([0.56128209, 1.84007049, 2.42080858, 0.79250474])

In [10]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>0.8800):
        print("\n Reached 88.00% accuracy so cancelling training!")
        self.model.stop_training = True

In [11]:
lr = [0.0005]
callbacks = myCallback()
for i in lr:
  print("Learning Rate: " + str(i))
  with strategy.scope():
    model = model_create()
    model.compile(Adam(learning_rate=i), loss='categorical_crossentropy', 
              metrics=['accuracy', 'AUC',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])
    
  history = model.fit(train_batches, steps_per_epoch = 512, epochs = 12, 
            validation_data = valid_batches, validation_steps = 128, 
             shuffle = True, callbacks = [callbacks])

model.save("/content/drive/My Drive/Eye Disease Research/model_vgg16-07292020-ver3.01.h5")
print("Saved model to disk")
model_json = model.to_json()
with open("/content/drive/My Drive/Eye Disease Research/model_vgg16_jsn-07292020-ver3.01.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Eye Disease Research/model_vgg16_wt-07292020-ver3.01.h5")
print("Saved model to disk")
  

Learning Rate: 0.0005
58892288/58889256 [==============================] - 3s 0us/step
Epoch 1/12
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


UnavailableError: ignored

In [ ]:
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy Using Training and Validation Data')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valididation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss Using Training and Validation Data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for AUC
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('AUC Using Training and Validation Data')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


Not needed
```
hist = model.fit(train_batches, steps_per_epoch = 512, epochs = 2, 
            validation_data = valid_batches, validation_steps = 128, 
            class_weight = class_weights, shuffle = True)

print(hist.history.keys())
# summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy Using Training and Validation Data for the Additional 2 Epochs')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valididation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss Using Training and Validation Data for the Additional 2 Epochs')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for AUC
plt.plot(hist.history['auc_1'])
plt.plot(hist.history['val_auc_1'])
plt.title('AUC Using Training and Validation Data for the Additional 2 Epochs')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
```

In [ ]:
class myCallback1(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>0.9100):
        print("\n Reached 92.0% accuracy so cancelling training!")
        self.model.stop_training = True

callback1 = myCallback1()
history_1 = model.fit(valid_batches, steps_per_epoch = 523, epochs = 1, 
            validation_data = test_batches, validation_steps = 32, 
            class_weight = class_weights, shuffle = True, callbacks = [callback1])
model.save("/content/drive/My Drive/Eye Disease Research/model_vgg16-07302020-ver3.01.h5")
print("Saved model to disk")
model_json = model.to_json()
with open("/content/drive/My Drive/Eye Disease Research/model_vgg16_jsn-07302020-ver3.01.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Eye Disease Research/model_vgg16_wt-07302020-ver3.01.h5")
print("Saved model to disk")

In [ ]:
print(history_1.history.keys())
# summarize history for accuracy
plt.plot(history_1.history['accuracy'])
plt.plot(history_1.history['val_accuracy'])
plt.title('Model Accuracy After Training on Validation Data and Testing on Test data')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history_1.history['loss'])
plt.plot(history_1.history['val_loss'])
plt.title('Model Loss After Training on Validation Data and Testing on Test data')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for AUC
plt.plot(history_1.history['auc'])
plt.plot(history_1.history['val_auc'])
plt.title('AUC Using Validation and Test Data')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.fit(valid_batches, steps_per_epoch = 523, epochs = 2, 
            validation_data = test_batches, validation_steps = 32, 
            class_weight = class_weights, shuffle = True, callbacks = [callback1])

## **Evaluate model on Test Data:**

In [ ]:
# Predict the accuracy on the Test Test!
acc = model.evaluate_generator(test_batches, steps=len(test_batches), verbose=1)
print("Model Accuracy on Test Data", acc[1]*100)

##**Predict Labels:**

In [ ]:
prediction = model.predict_generator(generator=test_batches, steps=len(test_batches), verbose=1)
print(np.argmax(prediction,axis=1))

## **Get Original Labels:**

In [ ]:
y = []
for x in range(0,len(test_batches)):
  for i in range(0,len(test_batches[x][1])):
    #print(test_batches[0][1][i])
    y.append(np.argmax(test_batches[x][1][i]))
print(len(y))

## **Plot Confusion Matrix:**

In [ ]:
con_mat = tf.math.confusion_matrix(labels=y, predictions=np.argmax(prediction,axis=1)).numpy()
print(con_mat)